# Workshop Azure Databricks
## 08. Cross Validation
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## Using Cross Validation

In this exercise, you will use cross-validation to optimize parameters for a regression model.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [0]:
import datetime
now = datetime.datetime.now()
print(now)

2021-02-10 13:55:06.888866

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Load the source data
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

data = data.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay", col("ArrDelay").alias("label"))

# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

### Define the Pipeline
Now define a pipeline that creates a feature vector and trains a regression model

In [0]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")

featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")

lr = LinearRegression(labelCol="label", featuresCol="features")

pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])

### Tune Parameters
You can tune parameters to find the best model for your data. <br>
To do this you can use the  **CrossValidator** class to evaluate each combination of parameters defined in a **ParameterGrid** against multiple *folds* of the data split into training and validation datasets, in order to find the best performing parameters. <br> 
> Note that this can take a long time to run because every parameter combination is tried multiple times.

In [0]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
cv = CrossValidator(estimator=pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=2)

model = cv.fit(train)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

### Test the Model
Now you're ready to apply the model to the test data.

In [0]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "label")

predicted.show()

+--------------------+-------------------+-----+
 features| prediction|label|
+--------------------+-------------------+-----+
[25.0,2.0,10.0,11...| 26.85974451920204| 30|
[25.0,2.0,10.0,49...| 1.4869270932998475| -12|
[25.0,2.0,10.0,37...| 16.171188559324257| 86|
[25.0,2.0,10.0,37...| 55.01903725136846| 55|
[25.0,2.0,10.0,37...|-1.2951273916215484| -10|
[25.0,2.0,10.0,37...| -2.386323199402007| -19|
[25.0,2.0,10.0,23...| 57.797654081963586| 50|
[25.0,2.0,10.0,13...| 2.9513470883801816| 37|
[25.0,2.0,10.0,48...| -9.599651900786185| -14|
[25.0,2.0,10.0,12...| 2.1747294731830706| -28|
[25.0,2.0,10.0,17...| 63.405642178994114| 29|
[25.0,2.0,10.0,0....| 58.97288315719359| 36|
[25.0,2.0,10.0,64...| 60.857951529680065| 53|
[25.0,2.0,10.0,29...| -6.436688839365843| -24|
[25.0,2.0,2.0,34....| 16.682997449678638| 6|
[25.0,2.0,2.0,11....| -7.318722825303382| -11|
[25.0,2.0,2.0,18....| 49.99699319998085| 26|
[25.0,2.0,2.0,23....| 70.84438437508778| 56|
[25.0,2.0,2.0,3.0...|0.24430885213275388| -10|
[25.0,2.0,2.0,3.0...| 46.31341139910788| 38|
+--------------------+-------------------+-----+
only showing top 20 rows

### Examine the Predicted and Actual Values
You can plot the predicted values against the actual values to see how accurately the model has predicted. In a perfect model, the resulting scatter plot should form a perfect diagonal line with each predicted value being identical to the actual value - in practice, some variance is to be expected.
Run the cells below to create a temporary table from the **predicted** DataFrame and then retrieve the predicted and actual label values using SQL. You can then display the results as a scatter plot, specifying **-** as the function to show the unaggregated values.

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
%sql
SELECT label, prediction FROM regressionPredictions

label,prediction
30,26.85974451920204
-12,1.4869270932998475
86,16.171188559324257
55,55.01903725136846
-10,-1.2951273916215484
-19,-2.386323199402007
50,57.797654081963586
37,2.9513470883801816
-14,-9.599651900786185
-28,2.1747294731830706


### Retrieve the Root Mean Square Error (RMSE)
There are a number of metrics used to measure the variance between predicted and actual values. Of these, the root mean square error (RMSE) is a commonly used value that is measured in the same units as the prediced and actual values - so in this case, the RMSE indicates the average number of minutes between predicted and actual flight delay values. You can use the **RegressionEvaluator** class to retrieve the RMSE.

In [0]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print("Root Mean Square Error (RMSE)=", "{:.6}".format(rmse))

Root Mean Square Error (RMSE)= 17.3129

> You can open Lab09